# Preparación de datos para Sagemaker Autopilot

- Usaremos la base de datos de puntuaciones de vinos.
- P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009. https://bit.ly/3enCZUz
- Este notebook la descarga y nos deja los csvs en s3.

In [1]:
import pandas as pd
import os

In [2]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'module_6/part_1'


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
local_prefix='winequality'
os.makedirs(local_prefix, exist_ok=True)

In [6]:
!wget --no-check-certificate https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv -P {local_prefix}

--2024-05-24 14:10:15--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
  Self-signed certificate encountered.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘winequality/winequality-white.csv’

winequality-white.c     [   <=>              ] 258.23K   431KB/s    in 0.6s    

2024-05-24 14:10:17 (431 KB/s) - ‘winequality/winequality-white.csv’ saved [264426]



In [7]:
from sklearn.model_selection import train_test_split
df = pd.read_csv(f'{local_prefix}/winequality-white.csv', delimiter = ';')

In [8]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [9]:
df_train, df_test = train_test_split(df, test_size = 0.1, random_state = 42)

In [10]:
df_train.shape, df_test.shape

((4408, 12), (490, 12))

In [11]:
df_train['quality'].value_counts()

quality
6    1983
5    1313
7     786
8     159
4     145
3      17
9       5
Name: count, dtype: int64

In [12]:
df_train.to_csv(f'{local_prefix}/winequality-white-train.csv', index=False)
df_test.to_csv(f'{local_prefix}/winequality-white-test.csv', index=False)

In [13]:
sagemaker.s3.S3Uploader.upload(local_path=f'{local_prefix}/winequality-white-train.csv',
                               desired_s3_uri=f's3://{bucket}/{prefix}/{local_prefix}',
                               sagemaker_session=sess)

's3://sagemaker-eu-west-1-467432373215/module_6/part_1/winequality/winequality-white-train.csv'

In [14]:
sagemaker.s3.S3Uploader.upload(local_path=f'{local_prefix}/winequality-white-test.csv',
                               desired_s3_uri=f's3://{bucket}/{prefix}/{local_prefix}',
                               sagemaker_session=sess)

's3://sagemaker-eu-west-1-467432373215/module_6/part_1/winequality/winequality-white-test.csv'